In [21]:
#bulk insert api reference: https://elasticsearch-py.readthedocs.io/en/7.x/helpers.html
from elasticsearch import Elasticsearch
host = "http://localhost:9200"
user = "elastic"
password = "password"
client = Elasticsearch(host, basic_auth=(user, password))

In [22]:
import json
#json file is a list of json instead of standard json format
data = [json.loads(line) for line in open("./shakespeare.json", "rb")]
types = []
for i in range(0, len(data), 2):
    type = data[i]["index"].get("_type")
    if type not in types:
        types.append(type)
print(types)


['act', 'scene', 'line']


In [24]:
index_name = "shakespeare"
client.indices.delete(index=index_name, ignore=[400, 404])
client.indices.create(index=index_name, ignore=400)

/tmp/ipykernel_6329/72972983.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.delete(index=index_name, ignore=[400, 404])
/tmp/ipykernel_6329/72972983.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.delete(index=index_name, ignore=[400, 404])
/tmp/ipykernel_6329/72972983.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.create(index=index_name, ignore=400)
/tmp/ipykernel_6329/72972983.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/ela

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'shakespeare'})

In [27]:
from elasticsearch import helpers
def generate_doc(file_path):
    data = [json.loads(line) for line in open(file_path, "rb")]
    for i in range(0, len(data), 2):
        type = data[i].get("index").get("_type")
        yield {
        "_index": "shakespeare-{}".format(type),
        "_id": data[i].get("index").get("_id"),
        "_source": data[i+1]
            }

helpers.bulk(client, actions=generate_doc("./shakespeare.json"))


/tmp/ipykernel_6329/642872350.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(client, actions=generate_doc("./shakespeare.json"))


(111396, [])

In [35]:
#try with the python client
#reference link: https://www.elastic.co/blog/a-practical-introduction-to-elasticsearch

#search in scene wich have play_name is Antony
type = "scene"
query = {
    "query":{
        "match" : {
            "play_name" : "Antony"
        }
    }
}
client.search(index="shakespeare-{}".format(type), query=query["query"])


/tmp/ipykernel_6329/2166020937.py:24: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.search(index="shakespeare-{}".format(type), query=query["query"])


ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': 9.882013, 'hits': [{'_index': 'shakespeare-scene', '_type': '_doc', '_id': '18633', '_score': 9.882013, '_source': {'line_id': 18634, 'play_name': 'Antony and Cleopatra', 'speech_number': 16, 'line_number': '1.1.62', 'speaker': 'DEMETRIUS', 'text_entry': 'Is Caesar with Antonius prized so slight?'}}, {'_index': 'shakespeare-scene', '_type': '_doc', '_id': '18637', '_score': 9.882013, '_source': {'line_id': 18638, 'play_name': 'Antony and Cleopatra', 'speech_number': 18, 'line_number': '1.1.66', 'speaker': 'DEMETRIUS', 'text_entry': 'I am full sorry'}}, {'_index': 'shakespeare-scene', '_type': '_doc', '_id': '18638', '_score': 9.882013, '_source': {'line_id': 18639, 'play_name': 'Antony and Cleopatra', 'speech_number': 18, 'line_number': '1.1.67', 'speaker': 'DEMETRIUS', 'text_entry': 'That he approves the c

In [36]:
#search in scene index which play_name is antony and speaker is Demetrius
type = "scene"
query = {
    "query":{
        "bool": {
            "must" : [
                {
                    "match" : {
                        "play_name" : "Antony"
                    }
                },
                {
                    "match" : {
                        "speaker" : "Demetrius"
                    }
                }
            ]
        }
    }
}
client.search(index="shakespeare-{}".format(type), query=query["query"])

/tmp/ipykernel_6329/288587973.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.search(index="shakespeare-{}".format(type), query=query["query"])


ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': 9.882013, 'hits': [{'_index': 'shakespeare-scene', '_type': '_doc', '_id': '18633', '_score': 9.882013, '_source': {'line_id': 18634, 'play_name': 'Antony and Cleopatra', 'speech_number': 16, 'line_number': '1.1.62', 'speaker': 'DEMETRIUS', 'text_entry': 'Is Caesar with Antonius prized so slight?'}}, {'_index': 'shakespeare-scene', '_type': '_doc', '_id': '18637', '_score': 9.882013, '_source': {'line_id': 18638, 'play_name': 'Antony and Cleopatra', 'speech_number': 18, 'line_number': '1.1.66', 'speaker': 'DEMETRIUS', 'text_entry': 'I am full sorry'}}, {'_index': 'shakespeare-scene', '_type': '_doc', '_id': '18638', '_score': 9.882013, '_source': {'line_id': 18639, 'play_name': 'Antony and Cleopatra', 'speech_number': 18, 'line_number': '1.1.67', 'speaker': 'DEMETRIUS', 'text_entry': 'That he approves the c

In [ ]:
{
    "size":0,
    "aggs" : {
        "Total plays" : {
            "cardinality" : {
                "field" : "play_name.keyword"
            }
        }
    }
}